# <br> Up your ML game 
![title](levelup.jpg)
## PyHEP DEV 
### Liv Våge 28.10.2025 

# Why this talk?

The ML ecosystem changes **faster than your average PhD**. 

By the time you finish reading the PyTorch docs, three new frameworks have been released and LLMs are writing better code than you.

**Goal:** Help you navigate the chaos and pick the right tools without spending 6 months on Stack Overflow.

---

## What we'll cover:
1. **ML Frameworks** - PyTorch, JAX, Keras, etc. (which one and why?)
2. **Workflow Tools** - W&B, MLflow, Optuna (stop using print statements)
3. **Training & Deployment** - hls4ml, ONNX, HTCondor (get off your laptop)
4. **HEP-ML Bridge** - uproot, awkward, hist (because ROOT files aren't going anywhere)
5. **Industry Tools** - What industry does better (and what we can steal)
6. **Fun Shortcuts** - LLMs, Hugging Face, and other "cheats"


# Common ML Frameworks

## The eternal question: Which framework should I use?

**Short answer:** PyTorch (probably)

**Long answer:** Let's actually compare them... 

### Quick Framework Comparison

| Framework | Best For | Pros | Cons |
|-----------|----------|------|------|
| **PyTorch** | Research, flexibility, HEP | Pythonic, great debugging, huge community | Verbose, more boilerplate |
| **PyTorch Lightning** | Production, clean code | Organized, less boilerplate, built-in best practices | Another abstraction to learn |
| **JAX** | Speed demons, researchers | FAST, functional programming, auto-vectorization | Functional paradigm learning curve |
| **Keras** | Beginners, quick prototypes | Super simple API, fast to start | Less flexibility, slower development |
| **Scikit-learn** | Classical ML, baselines | Easy, stable, great docs | Not for deep learning |
| **XGBoost** | Tabular data, structured features | Fast, interpretable, great for HEP kinematics | Not for complex deep learning |

**Decision tree:**
- Just starting? → **Keras** or **Scikit-learn**
- Need a quick baseline on tabular data? → **XGBoost** or **Scikit-learn**
- Working with HEP kinematic features? → **XGBoost** (often best!)
- Doing research/custom architectures? → **PyTorch**
- Want cleaner code? → **PyTorch Lightning**
- Need maximum speed? → **JAX**
- Working in a team? → **PyTorch** or **PyTorch Lightning**


In [1]:
# Quick demo: Same simple neural network in different frameworks
import numpy as np

# Generate some fake HEP-like data (4 kinematic features)
np.random.seed(42)
X_train = np.random.randn(1000, 4).astype(np.float32)
y_train = (X_train[:, 0] + X_train[:, 1] > 0).astype(np.float32)
X_test = np.random.randn(200, 4).astype(np.float32)
y_test = (X_test[:, 0] + X_test[:, 1] > 0).astype(np.float32)

print(f"Training data: {X_train.shape}, Labels: {y_train.shape}")


Training data: (1000, 4), Labels: (1000,)


#### Option 1: PyTorch (the verbose way)


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Define model
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.layers(x)

# Setup
model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert to PyTorch tensors
X_train_t = torch.from_numpy(X_train)
y_train_t = torch.from_numpy(y_train).unsqueeze(1)

# Training loop (the part everyone copy-pastes)
model.train()
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()
    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

print("✅ PyTorch training done!")


Epoch 0, Loss: 0.7555
Epoch 5, Loss: 0.7462
✅ PyTorch training done!


#### Option 2: Scikit-learn (the "I just want it to work" way)


In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# That's it. Literally.
clf = MLPClassifier(hidden_layer_sizes=(16,), max_iter=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print("✅ Scikit-learn: 3 lines and done. Beginners rejoice!")


Accuracy: 0.980
✅ Scikit-learn: 3 lines and done. Beginners rejoice!


/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


#### Option 3: PyTorch Lightning (the "I want my code to not be a mess" way)


In [4]:
# Commented out - install if you want to run:
# !pip install pytorch-lightning

# import pytorch_lightning as pl
#
# class LitModel(pl.LightningModule):
#     def __init__(self):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(4, 16), nn.ReLU(), nn.Linear(16, 1), nn.Sigmoid()
#         )
#     
#     def forward(self, x):
#         return self.layers(x)
#     
#     def training_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         loss = nn.BCELoss()(y_hat, y)
#         self.log('train_loss', loss)
#         return loss
#     
#     def configure_optimizers(self):
#         return optim.Adam(self.parameters(), lr=0.001)
#
# # Automatic logging, checkpointing, multi-GPU support, etc.
# # trainer = pl.Trainer(max_epochs=10)
# # trainer.fit(model, train_dataloader)

print("💡 Lightning = PyTorch + Organization + Built-in Best Practices")


💡 Lightning = PyTorch + Organization + Built-in Best Practices


### JAX: For when you need SPEED

JAX is functional programming meets deep learning. It's fast. Really fast.

**Key features:**
- `jax.jit` - Just-In-Time compilation (makes code go brrrr)
- `jax.grad` - Automatic differentiation of anything
- `jax.vmap` - Auto-vectorization
- Works on GPU/TPU with zero code changes

**Warning:** You have to think differently (functional programming, immutability)


### XGBoost: The Gradient Boosting Powerhouse

**What it is:** Extreme Gradient Boosting - tree-based ensemble method

**Why it matters for HEP:**
- Handles tabular data exceptionally well (which HEP has lots of!)
- Often outperforms neural networks on structured data
- Interpretable (feature importance, SHAP values)
- Fast training and inference
- Great baseline before trying deep learning

**When to use:**
- Tabular data with many features
- Need quick, interpretable results
- Want feature importance
- Limited training data available


In [ ]:
# XGBoost example
# Uncomment if you have xgboost installed
# !pip install xgboost

try:
    import xgboost as xgb
    from sklearn.metrics import accuracy_score
    
    # Train XGBoost
    clf_xgb = xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
    clf_xgb.fit(X_train, y_train)
    
    # Evaluate
    y_pred_xgb = clf_xgb.predict(X_test)
    acc_xgb = accuracy_score(y_test, y_pred_xgb)
    print(f"XGBoost Accuracy: {acc_xgb:.3f}")
    
    # Show feature importance
    print("\nFeature Importance:")
    for i, imp in enumerate(clf_xgb.feature_importances_):
        print(f"  Feature {i}: {imp:.3f}")
except ImportError:
    print("XGBoost not installed. Install with: pip install xgboost")
    print("\nXGBoost is great for:")
    print("- Tabular/structured data")
    print("- Quick benchmarks")
    print("- Understanding feature importance")


### 1-to-1 Comparison: Timing and Accuracy

Let's see how different frameworks perform on the exact same data:


In [ ]:
# Comprehensive comparison of frameworks
import time
import pandas as pd

results = []

# Test PyTorch
start = time.time()
model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
X_train_t = torch.from_numpy(X_train)
y_train_t = torch.from_numpy(y_train).unsqueeze(1)

model.train()
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()

# Evaluate
model.eval()
with torch.no_grad():
    X_test_t = torch.from_numpy(X_test)
    predictions = model(X_test_t).numpy().flatten() > 0.5
    acc = (predictions == y_test).mean()

pytorch_time = time.time() - start
results.append(('PyTorch', pytorch_time, acc))

# Test Scikit-learn
start = time.time()
clf_skl = MLPClassifier(hidden_layer_sizes=(16,), max_iter=100, random_state=42)
clf_skl.fit(X_train, y_train)
acc_skl = clf_skl.score(X_test, y_test)
sklearn_time = time.time() - start
results.append(('Scikit-learn', sklearn_time, acc_skl))

# Test XGBoost (if available)
try:
    import xgboost as xgb
    start = time.time()
    clf_xgb = xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss', verbosity=0)
    clf_xgb.fit(X_train, y_train)
    acc_xgb = clf_xgb.score(X_test, y_test)
    xgb_time = time.time() - start
    results.append(('XGBoost', xgb_time, acc_xgb))
except:
    pass

# Display results
df = pd.DataFrame(results, columns=['Framework', 'Time (s)', 'Accuracy'])
df = df.sort_values('Time (s)')
print("Framework Comparison Results:\n")
print(df.to_string(index=False))
print(f"\nFastest: {df.iloc[0]['Framework']} ({df.iloc[0]['Time (s)']:.3f}s)")
print(f"Most Accurate: {df.loc[df['Accuracy'].idxmax(), 'Framework']} ({df['Accuracy'].max():.3f})")


/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


KeyError: 'Time'

In [8]:
# JAX quick demo - speed comparison
# Commented out - install jax if you want to run

# import jax
# import jax.numpy as jnp
# from jax import jit, grad
# import time
#
# # Regular function
# def slow_function(x):
#     return jnp.sum(x ** 2)
#
# # JIT-compiled version (fast!)
# fast_function = jit(slow_function)
#
# x = jnp.ones((10000,))
# 
# # First call compiles, subsequent calls are FAST
# %timeit slow_function(x).block_until_ready()
# %timeit fast_function(x).block_until_ready()

print("⚡ JAX can be 10-100x faster for some operations")
print("📚 Good resources: JAX docs, Google Colab tutorials")


⚡ JAX can be 10-100x faster for some operations
📚 Good resources: JAX docs, Google Colab tutorials


# ML Workflow Tools

## Stop using `print()` for everything!

You know you've done this:
```python
print(f"Epoch {epoch}, loss: {loss}, acc: {acc}, lr: {lr}, ...")
# *scrolls through terminal for 10 minutes*
```

**There's a better way.**

### Weights & Biases (W&B) - The Gold Standard

**What it does:**
- Automatic logging of metrics, hyperparameters, system info
- Beautiful dashboards
- Experiment comparison
- Model versioning
- Artifact tracking
- **Free for academics!**

**When to use:** Any serious project. Just use it.


In [ ]:
# W&B Quick Start (commented - needs account)
# !pip install wandb

# import wandb
#
# # Initialize
# wandb.init(project="pyhep-demo", name="experiment-1")
#
# # Log whatever you want
# for epoch in range(10):
#     loss = 1.0 / (epoch + 1)  # fake loss
#     wandb.log({"loss": loss, "epoch": epoch})
#
# # Log your model, datasets, anything!
# wandb.finish()

print("🎨 W&B gives you beautiful dashboards without the matplotlib pain")
print("💡 Pro tip: wandb.watch(model) tracks gradients automatically!")


### MLflow - The Open Source Alternative

**Pros:**
- Fully open source
- Self-hosted (for the privacy-conscious)
- Experiment tracking + model registry
- Works with any ML library

**Cons:**
- Less pretty than W&B
- Need to host it yourself
- Smaller community

**When to use:** You need full control, can't/won't use cloud services


In [ ]:
# MLflow example
# import mlflow
#
# mlflow.start_run()
# mlflow.log_param("learning_rate", 0.001)
# mlflow.log_metric("accuracy", 0.95)
# mlflow.log_artifact("model.pth")
# mlflow.end_run()

print("🏠 MLflow: Great for on-premise setups")


### Optuna - Hyperparameter Optimization Made Easy

**Stop doing grid search in 2025!**

Optuna uses smart algorithms (TPE, CMA-ES) to find good hyperparameters faster.


In [ ]:
# Optuna example - hyperparameter tuning
# !pip install optuna

# import optuna
#
# def objective(trial):
#     # Suggest hyperparameters
#     lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
#     n_layers = trial.suggest_int("n_layers", 1, 3)
#     
#     # Train your model with these hyperparameters
#     # ... training code ...
#     # accuracy = train_and_evaluate(lr, n_layers)
#     
#     # Return metric to optimize
#     # return accuracy
#     return 0.95  # dummy
#
# # Optimize!
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100)
#
# print(f"Best params: {study.best_params}")

print("🎯 Optuna: Smarter than grid search, easier than manual tuning")
print("💡 Integrates with W&B, PyTorch Lightning, etc.")


### Workflow Tools: Quick Comparison

| Tool | Best For | Setup Difficulty | Cost |
|------|----------|------------------|------|
| **W&B** | Everything | Easy | Free (academic) |
| **MLflow** | On-premise, privacy | Medium | Free (self-host) |
| **Optuna** | Hyperparameter tuning | Easy | Free |
| **b-hive** | CERN users | Easy | Free (CERN) |

**Pro tip:** Use W&B + Optuna together. They integrate perfectly!


# Model Training & Deployment

## Get off your laptop!

Your MacBook is crying. Let's talk about scaling up.

### HTCondor - The HEP Classic

**What it is:** Distributed computing system used at CERN and beyond

**Pros:**
- Already set up at most HEP institutions
- Handle thousands of jobs
- Free (for you)

**Cons:**
- Not designed for ML (but works!)
- Can be slow to start
- Queue times vary

**When to use:** You're at a HEP institution and need to run many jobs

```bash
# Example HTCondor submit file
# universe = vanilla
# executable = train_model.sh
# arguments = --learning-rate 0.001
# queue 100 # Submit 100 jobs!
```


### SWAN - CERN's Jupyter Hub

**What it is:** Cloud-based Jupyter notebooks at CERN

**Pros:**
- Access to CERN data
- Pre-configured environment
- Spark integration
- GPUs available

**When to use:** You're at CERN and want to prototype quickly

**URL:** https://swan.cern.ch/


### ONNX - Make your model portable

**Problem:** Trained in PyTorch, but production uses TensorFlow (or C++, or...)

**Solution:** ONNX (Open Neural Network Exchange)

**What it does:**
- Convert models between frameworks
- Optimize for inference
- Deploy anywhere (edge devices, web, etc.)


In [ ]:
# ONNX Example - Export PyTorch model
# import torch.onnx
#
# # Your trained PyTorch model
# dummy_input = torch.randn(1, 4)
# torch.onnx.export(model, dummy_input, "model.onnx")
#
# # Now use it in other frameworks!
# import onnxruntime as ort
# session = ort.InferenceSession("model.onnx")
# result = session.run(None, {"input": X_test[:1]})

print("📦 ONNX: Train anywhere, deploy everywhere")
print("🎯 Especially useful for edge deployment and production")


### hls4ml - ML on FPGAs

**The coolest HEP-specific tool you didn't know you needed**

**Problem:** You need ultra-low latency inference (< 1 microsecond) for triggers

**Solution:** hls4ml converts your neural network to FPGA firmware

**Use cases:**
- LHC trigger systems
- Real-time event selection
- Anything requiring hardware acceleration

```python
# import hls4ml
# config = hls4ml.utils.config_from_keras_model(model, granularity='name')
# hls_model = hls4ml.converters.convert_from_keras_model(
# model, hls_config=config, output_dir='my-hls-test'
# )
# hls_model.compile()
```

**Mind-blowing:** Your Python model → Hardware in < 1 hour


# HEP-ML Bridge Tools

## The "ROOT files aren't going anywhere" section

You can't do ML without data. In HEP, that means ROOT files, weird event structures, and ragged arrays.

**These tools save your sanity:**

### uproot - Read ROOT files without ROOT

**The game changer.**

Before: Install ROOT, fight with Python bindings, cry 
After: `pip install uproot`, read files with pandas-like syntax

**No C++ dependencies. No ROOT installation. Pure Python bliss.**


In [ ]:
# uproot example (without actual ROOT file)
# import uproot
#
# # Read ROOT file
# file = uproot.open("data.root")
# tree = file["Events"]
#
# # Get branches as arrays
# pt = tree["jet_pt"].array()
# eta = tree["jet_eta"].array()
#
# # Or as pandas DataFrame
# df = tree.arrays(["jet_pt", "jet_eta", "jet_phi"], library="pd")

print("🎉 uproot: Because life's too short to compile ROOT")
print("💡 Works with awkward, numpy, pandas, and more!")


### Awkward Array - Handle Jagged Data

**Problem:** HEP events have variable-length lists (jets, tracks, etc.)

**Standard approach:** Pad everything, waste memory, write ugly code

**Awkward Array:** Numpy for jagged/nested/variable-length data


In [ ]:
import awkward as ak

# Events with variable numbers of jets
events = ak.Array([
    {"jets": [{"pt": 50, "eta": 0.1}, {"pt": 30, "eta": -0.5}]},  # 2 jets
    {"jets": [{"pt": 100, "eta": 1.2}]},                           # 1 jet
    {"jets": [{"pt": 40, "eta": 0.3}, {"pt": 35, "eta": 0.8}, {"pt": 25, "eta": -1.0}]}  # 3 jets
])

# Operations work naturally on jagged data!
jet_pts = events.jets.pt
print("Jet pts:", jet_pts)

# Calculate things per event
leading_jet_pt = ak.max(events.jets.pt, axis=1)
print("Leading jet pt per event:", leading_jet_pt)

# Slice like numpy
high_pt_jets = events.jets[events.jets.pt > 35]
print("High-pt jets:", high_pt_jets)

print("\n✨ Awkward: No more padding! No more for-loops!")


### hist - Modern Histogramming

**ROOT's TH1/TH2 are... showing their age.**

`hist` is a modern, Pythonic histogramming library:
- Clean syntax
- Integrates with numpy, awkward
- Beautiful plotting with matplotlib/mplhep
- Type hints, named axes, units


In [ ]:
# Modern histogramming with hist
# !pip install hist

# from hist import Hist
# import hist
#
# # Create histogram with named axes
# h = Hist.new.Reg(50, 0, 200, name="pt", label="$p_T$ [GeV]").Double()
# h.fill(pt=np.random.exponential(50, 10000))
#
# # Plot (works with matplotlib)
# import matplotlib.pyplot as plt
# h.plot()
# plt.show()

print("📊 hist: Histograms that don't make you want to cry")
print("💡 Named axes, units, better plotting. Just better.")


### The Complete HEP-ML Pipeline

```python
import uproot
import awkward as ak
import numpy as np
from hist import Hist

# 1. Read ROOT file
with uproot.open("data.root:Events") as tree:
 events = tree.arrays(["jet_*"], library="ak")

# 2. Process with awkward
good_events = events[ak.num(events.jet_pt) >= 2]
leading_jets = good_events.jet_pt[:, 0]

# 3. Make histograms
h = Hist.new.Reg(50, 0, 200, name="pt").Double()
h.fill(leading_jets)

# 4. Convert to ML format
X = ak.to_numpy(ak.pad_none(events.jet_pt, 5, clip=True))
# Now feed to PyTorch/JAX/etc!
```

**The dream: ROOT file → ML model in < 50 lines**


# Industry Tools

## What industry does better (and what we can steal)

HEP is amazing at physics. Industry is amazing at software engineering.

**Let's learn from them:**

### Testing & Linting - Stop Breaking Things

**Industry:** Comprehensive tests, CI/CD, code review, linting 
**HEP:** "It worked on my machine" 

**Tools you should use:**

1. **pytest** - Testing framework
2. **black** - Code formatter (stop arguing about formatting)
3. **ruff** - Fast linter
4. **mypy** - Type checking
5. **pre-commit** - Run checks before committing


In [ ]:
# Quick testing example
# tests/test_model.py

# import pytest
# import torch
#
# def test_model_output_shape():
#     model = SimpleNN()
#     x = torch.randn(10, 4)
#     output = model(x)
#     assert output.shape == (10, 1), "Wrong output shape!"
#
# def test_model_output_range():
#     model = SimpleNN()
#     x = torch.randn(10, 4)
#     output = model(x)
#     assert torch.all(output >= 0) and torch.all(output <= 1), "Sigmoid broken!"

print("✅ Write tests. Your future self will thank you.")
print("💡 Pro tip: Test your preprocessing! That's where most bugs hide.")


### GitHub Actions - Automate Everything

**Stop manually running tests. Let robots do it.**

Example `.github/workflows/test.yml`:
```yaml
name: Tests
on: [push, pull_request]
jobs:
 test:
 runs-on: ubuntu-latest
 steps:
 - uses: actions/checkout@v3
 - uses: actions/setup-python@v4
 with:
 python-version: '3.10'
 - run: pip install -r requirements.txt
 - run: pytest
 - run: ruff check .
```

**Now every commit is automatically tested. Magic!**


### AWS SageMaker - When You Need Industrial Scale

**What it is:** AWS's ML platform (training, deployment, everything)

**Pros:**
- Scales to infinity
- Managed infrastructure
- Production-ready deployment
- AutoML features

**Cons:**
- Costs money (sometimes a lot)
- Learning curve
- Vendor lock-in

**When to use:** 
- You need serious scale
- You have budget
- Production deployment

**HEP alternative:** Usually HTCondor + custom scripts (cheaper, less polished)


### What HEP Can Learn from Industry

| Practice | Industry | HEP | What to do |
|----------|----------|-----|-----------|
| **Testing** | Comprehensive | Sparse | Write pytest tests! |
| **CI/CD** | GitHub Actions | Manual | Add GitHub Actions |
| **Code Review** | Required | Optional | Make PRs mandatory |
| **Documentation** | Detailed | "See code" | Write docstrings |
| **Versioning** | Semantic | Git SHA | Use proper versions |
| **Linting** | Enforced | What's that? | Use ruff/black |

**Bottom line:** Treat your code like a product, not a script.


# Fun Shortcuts & "Cheats"

## Work smarter, not harder

The secret sauce. The shortcuts your supervisor doesn't want you to know about.

### 1. Make LLMs Do Your Work

**Let's be honest:** We're all using ChatGPT/Claude/Copilot

**Good uses:**
- Boilerplate code (data loaders, training loops)
- Documentation and docstrings
- Bug finding
- Code explanation
- Unit test generation

**Bad uses:**
- Novel research code (they hallucinate)
- Critical analysis code (verify everything!)
- Anything you don't understand

**Pro tips:**
- Be specific: "Write a PyTorch data loader for awkward arrays"
- Iterate: Start simple, add complexity
- **Always understand the code it gives you**


In [ ]:
# Example: Let's ask an LLM to write a custom loss function
# Prompt: "Write a PyTorch loss function that combines binary cross entropy with a custom regularization term"

# LLM output (cleaned up):
class CustomLoss(nn.Module):
    def __init__(self, reg_weight=0.01):
        super().__init__()
        self.bce = nn.BCELoss()
        self.reg_weight = reg_weight
    
    def forward(self, predictions, targets, model_params):
        bce_loss = self.bce(predictions, targets)
        # L2 regularization
        reg_loss = sum(p.pow(2.0).sum() for p in model_params)
        return bce_loss + self.reg_weight * reg_loss

print("🤖 LLMs: Your 24/7 coding assistant")
print("⚠️  But verify everything! They confidently hallucinate.")


### 2. Steal from Hugging Face

**Hugging Face:** GitHub for ML models

**What's there:**
- 500,000+ pre-trained models
- Datasets
- Code examples
- Entire pipelines

**You can:**
- Fine-tune existing models (faster than training from scratch)
- Use pre-trained embeddings
- Copy architectures
- Download datasets


In [ ]:
# Hugging Face example - Use a pre-trained model
# !pip install transformers

# from transformers import pipeline
#
# # Use a pre-trained model with ONE line
# classifier = pipeline("sentiment-analysis")
# result = classifier("The LHC is amazing!")
# print(result)

# For HEP: Look for:
# - Transformer models for jet tagging
# - Graph neural networks
# - Anomaly detection models

print("🤗 Hugging Face: Don't reinvent the wheel, fine-tune it!")
print("💡 Search for 'particle physics', 'HEP', 'jet tagging' on HF")


### 3. Quick Prototyping Tricks

**Trick 1: Use `fastai` for rapid prototyping**
- High-level API (even simpler than Keras)
- Best practices built-in
- Great for quick experiments

**Trick 2: `torchinfo` for model debugging**
```python
from torchinfo import summary
summary(model, input_size=(1, 4))
# Instantly see: layers, params, output shapes
```

**Trick 3: `einops` for tensor operations**
```python
from einops import rearrange, reduce
# No more confusing reshapes!
x = rearrange(x, 'b c h w -> b (c h w)')
```

**Trick 4: `timm` for vision models**
- 1000+ pre-trained computer vision models
- `pip install timm`


In [ ]:
# Quick tricks demo

# Trick: torchinfo for model summary
# !pip install torchinfo
# from torchinfo import summary
# summary(model, input_size=(32, 4))  # batch_size=32, features=4

# Trick: Use repr to see object details
print("Model architecture:")
print(SimpleNN())

# Trick: Quick timing
import time
start = time.time()
# ... your code ...
print(f"Took {time.time() - start:.3f}s")

# Better: Use %%time or %%timeit in Jupyter!

print("\n💡 Small tricks add up to big time savings!")


### 4. Dataset Shortcuts

**Don't start from scratch:**

1. **Papers with Code** - Find datasets and benchmarks
2. **Kaggle** - Tons of curated datasets
3. **UCI ML Repository** - Classic datasets
4. **HEP Data** - Published HEP datasets
5. **Zenodo** - Open science data

**For HEP specifically:**
- CERN Open Data Portal
- LHC Olympics datasets
- Public collision data

**Pro tip:** Start with a small subset! Debug on 1000 events, not 1M.


### 5. The Ultimate Shortcut List

**Must-bookmark resources:**

 **Learning:**
- fast.ai course (free, excellent)
- PyTorch tutorials (official)
- Kaggle Learn (interactive)
- Papers with Code (implementations)

🛠️ **Tools:**
- GitHub Copilot / Cursor (AI pair programmer)
- Paperswithcode.com (find state-of-the-art)
- Connected Papers (explore research)

💬 **Community:**
- PyHEP working group
- Scikit-HEP GitHub
- ML4Jets workshop materials
- Discord/Slack ML communities

🎓 **HEP-specific:**
- IML (Inter-experimental Machine Learning)
- ML4Jets workshops
- PyHEP workshops
- IRIS-HEP training


# Summary: Your ML Toolkit

## Quick Reference Guide

### For Beginners:
1. **Start here:** Scikit-learn for classical ML, Keras for deep learning
2. **Read data:** uproot for ROOT files
3. **Track experiments:** W&B (free for academics!)
4. **Learn:** fast.ai course, official PyTorch tutorials

### For Intermediate Users:
1. **Framework:** PyTorch or PyTorch Lightning
2. **Data:** uproot + awkward array
3. **Optimization:** Optuna
4. **Deployment:** ONNX
5. **Code quality:** pytest, ruff, GitHub Actions

### For Advanced Users:
1. **Speed:** JAX for compute-intensive tasks
2. **Scale:** HTCondor or cloud (SageMaker)
3. **Hardware:** hls4ml for FPGAs
4. **Tools:** Custom pipelines with all the above

### Universal Tips:
- Use version control (git)
- Write tests (pytest)
- Log experiments (W&B/MLflow)
- Document your code
- Start small, scale up
- Leverage pre-trained models (Hugging Face)
- Use LLMs wisely (verify everything!)


# Final Thoughts

## The ML landscape is vast, but you don't need to know everything

**Key takeaways:**

1. **Pick tools that fit YOUR needs** - Don't use fancy tools just because they're fancy
2. **Start simple, add complexity** - Scikit-learn → PyTorch → JAX
3. **Steal shamelessly** - Use pre-trained models, copy good code, ask LLMs
4. **Automate early** - W&B, GitHub Actions, testing save time in the long run
5. **Bridge HEP ↔ ML** - uproot, awkward, hist make life easier
6. **Learn from industry** - Testing, CI/CD, code quality matter
7. **Community is key** - PyHEP, ML4Jets, IML, Scikit-HEP

---

## Most important:

### **The best tool is the one you'll actually use.**

Perfect code that doesn't exist < Working code that's "good enough"

---

# Questions? 

Resources:
- These slides: [your-repo-link]
- Scikit-HEP: https://scikit-hep.org/
- PyHEP: https://hepsoftwarefoundation.org/workinggroups/pyhep.html
- My contact: [your-contact]

**Now go build something cool!** 
